## Evaluation Notebook

In [2]:
!python -m spacy download en_core_web_sm



[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
pip install spacy

In [1]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import re
import os
from spacy import displacy
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from spacy.util import filter_spans
 



FOODKEEPER_PATH = "datasets/FoodKeeper-Data.xls"
TRAINING_DATA_PATH = "datasets/data.csv"
MODEL_PATH = "output/model-last"
TEST_DATA_PATH = "datasets/test_data.csv"
REAL_TWITTER_DATA_PATH = "datasets/training1.6.csv"
#STARTING_KEYWORD_COUNT = 10
#TRAINING_LOOP_ITERATIONS = 3
#REQUIRED_KEYWORDS = 3

pd.options.mode.chained_assignment = None

#nlp = spacy.load(MODEL_PATH)
nlp = spacy.blank("en")
food_data = pd.read_excel(FOODKEEPER_PATH, sheet_name = "Product")
training_data = pd.read_csv(TRAINING_DATA_PATH,index_col = False, header = None)
test_data = pd.read_csv(TEST_DATA_PATH)
live_tweets = pd.read_csv(REAL_TWITTER_DATA_PATH, header = None)


#loop through and count the specific entities
keywords = [] #'chicken', 'milk', 'butter', 'cheese'
sampleData = []


    
#update rank tweet to take the counter as a parameter and condense both rankings
def rankTweet(tweet, model):
#     model = spacy.load(MODEL_PATH)
#     tweetKeywords = []
    doc = model(tweet)
    return len(doc.ents)
       
    
def findNewKeywords(tweet, keywords):
    foodkeeperKeys = foodKeeperInfo()
    x = tweet.split()
    word = ""
    i = 0
    while i < len(x):
    #for i in range(len(x)):
        z = 1
        if x[i] in foodkeeperKeys:
            word = x[i]
        try:
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in foodkeeperKeys: #keywords
                word = foundBiWord
                z = 2
        except:
            pass
        
        try:
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in foodkeeperKeys: #keywords:
                word = foundTriWord
                z = 3
        except:
            pass
        i += z
        
        if word not in keywords and word != "":
            keywords.append(word)
    return keywords
    

#Function to find the most common verbs in the tweets
def getCommonVerbs(data):
    import en_core_web_sm
    nlp2 = spacy.load("en_core_web_sm")
    count = 0
    myVerbs = {}
    for i in range(len(data[0])):
        doc = nlp2(data[0][i])
        for token in doc:
            if token.pos_ == "VERB":
                if token.text in myVerbs:
                    myVerbs[token.text] = myVerbs[token.text] + 1
                else:
                    if token.text not in nlp2.Defaults.stop_words:
                        myVerbs[token.text] = 1
        

    topVerbs = dict(sorted(myVerbs.items(), key = lambda item: item[1], reverse=True)[:10])
    return [key for key in topVerbs]
    

    
def convertToTrainingFormat(tweet, keywords):
    foodKeeperKeywordsTest = foodKeeperInfo()
    x = tweet.split()
    myEnts = {'entities':[]}
    found = False
    i = 0
    foundWords = []
    while i < len(x):
        z = 1
        newWord = ""
        if x[i] in keywords:
            pos = tweet.find(x[i])
            y = (pos, pos + len(x[i]), 'FOOD')
            found = True
        if x[i] in foodKeeperKeywordsTest:
            newWord = x[i]
            
        try:
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i])+len(x[i+1]) + 1, 'FOOD')
                found = True
                z = 2
            if foundBiWord in foodKeeperKeywordsTest:
                newWord = foundBiWord
        except:
            pass
        
        try:
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i])+len(x[i+1])+len(x[i+2]) + 2, 'FOOD')
                found = True
                z = 3
            if foundTriWord in foodKeeperKeywordsTest:
                newWord = foundTriWord
        except:
            pass
        
        try:
            if y not in myEnts['entities']:
                myEnts['entities'].append(y)    
        except:
            pass

        if newWord != "" and newWord not in keywordRanker:
            keywordRanker[newWord] = 1
        elif newWord != "" and newWord in keywordRanker:
            keywordRanker[newWord] += 1
        #print(z)
        i += z
        #print(i)
    formatted = (tweet, myEnts)
    #print(formatted)
    if found:
        return formatted
    else: 
        return ()
     
#Gathers all the keywords from the FoodKeeper database
def foodKeeperInfo():              
    keywords = []
    for word in food_data['Name']:
        word = word.replace(" or ", " ")
        word = re.sub('[/,]', ' ', word)
        word = word.lstrip()
        word = word.rstrip()

        if word.lower() not in keywords: 
            keywords.append(word.lower())

    #print("Total foodkeeper food names: " + str(len(keywords)))        
    #for element in sorted(keywords):
        #print(element)
        
    return keywords

foodKeeperKeywordsTest = foodKeeperInfo()


def preProcess(tweet):
    #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
    tweet = tweet.lower()
    tweet = re.sub('@[a-zA-z0-9]*', '', tweet)              # <USERNAME>
    tweet = re.sub('http[a-zA-z0-9./:]*', '', tweet)       # <URL>
    tweet = re.sub('[.,-]*', '', tweet)
    tweet = re.sub('&amp;', 'and', tweet)
    
    return tweet


noEntity= []

keywordRanker = {}

def trainModel(data):
    #Initialize all the variables
    keywords = [] #foodKeeperInfo()
    oldKeywords = []
    newKeywords = []
    
      
    
    #commonVerbs = getCommonVerbs(data) 
    print(commonVerbs)
    print("Common Verbs gathered...", '\n')
    
    #entityCheckCount controls how many entities are required to 
    #add a Tweet to be trained
    
    entityCheckCount = 3
    
    counter = 0
    trainingLoop = True
    
    while trainingLoop:
        counterText = "~~~~~~~~~~~~~~~~~"+str(counter)+"~~~~~~~~~~~~~~~~~"
        print(counterText)
        
        nlp = spacy.blank("en") # load a new spacy model
        try:
            model = spacy.load(MODEL_PATH)
            print('Model loaded...')
        except:
            print('No model...')
        
        db = DocBin() # create a DocBin object
        
        myTweets = []
        
        #Loop through all the tweets
        #This loop is necessary to get the most common keywords 
        #in the convertToTrainingFormat function
        
        for i in range(len(data[0])): #len(data[0])
            #useless if?
#             if i % 500 == 0:
#                 print(i)

            if counter == 0:
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                
            #If counter is 1 then there is no model to check so 
            #a word count is performed
            elif counter == 1:
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                if x!= ():
                    if len(x[1]['entities']) > entityCheckCount:
                        #print("Found tweet", x[0])
                        myTweets.append(x)  
            else:
                #Convert each tweet into spacy training format
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                checkPassed = False
                if x != ():
                    #Check the ranking of the tweet
                    if rankTweet(x[0], model) > entityCheckCount:
                        checkTweet = x[0].split()
                        
                        #Check to see if tweet has one of the common verbs
                        for word in checkTweet:
                            if word in commonVerbs:
                                checkPassed = True
                                
                        if True: #checkPassed:
                            #print("Checking rank...")
                            myTweets.append(x)
        
                        
#         Initialize the keywords
       
        
        if counter == 0:  
            # Set keywords to be all keywords found in foodkeeper
#            keywords = foodKeeperInfo()
            sortedKeywords =  sorted(keywordRanker, key=keywordRanker.get, reverse=True)

            for i in range(15): #sortedKeywords
                keywords.append(sortedKeywords[i])
            #print(sortedKeywords[i], keywordRanker[sortedKeywords[i]])
            

        elif counter > 0:
            for text, annot in tqdm(myTweets): # data in previous format
                doc = nlp.make_doc(text) # create doc object from text
                ents = []
                for start, end, label in annot["entities"]: # add character indexes
                    span = doc.char_span(start, end, label=label, alignment_mode="contract")
                    
                    if span is None:
                        print("Skipping entity")
                    elif ents == []: ents.append(span) 
                    else:
                        #Check to see if any entities are overlapping i.e rice and rice cakes
                        ents.append(span)
                        
                        
#                         for ent in ents:
#                             if ent is not None:
#                                 entLength = ent.end - ent.start
#                                 if span.start == ent.start or span.end == ent.end:
#                                     if entLength > (span.end - span.start):
#                                         continue
#                                     else:
#                                         ents.remove(ent)
#                                         ents.append(span)
#                                 else:
#                                     ents.append(span)
                    #if span not in ents: ents.append(span)
                            
                                #print(span.start, span.end, ents)
                    #put into for loop
#                 foundEnts = []
#                 newEnts = []
#                 for ent in ents:
#                     if ent.text not in foundEnts:
#                         foundEnts.append(ent.text)
#                         newEnts.append(ent)
                        
                newEnts = filter_spans(ents)              
                #try:
                doc.ents = newEnts # label the text with the ents
                    #print(doc)
                db.add(doc)
                #except:
                    #print("Error 10: ", doc)

            db.to_disk("./train.spacy") # save the docbin object
                
            #If problems are occuring with the models not appearing
            #ensure that the command is valid, specifically python is the correct
            #PATH variable name on your machine
            #--paths.train should be where the docbin object is saved
            stream = os.popen('python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy')
            print(stream.read())
            print("Total keywords: ", str(len(keywords)))
            print("List of Keywords:\n\n",keywords,"\n\n")
            oldKeywords = len(keywords)
            
            
            #Look for new keywords
            for element in myTweets:
                keywords = findNewKeywords(element[0], keywords)
                
            
            #No new keywords are found
            if (oldKeywords == len(keywords)) and counter > 1 and entityCheckCount != 1:
                entityCheckCount -= 1
                print("Decreasing entityCheckCount variable by 1")
                print("entityCheckCount = ", entityCheckCount)
                

            #New keywords are found and entity rank check == 1
            elif (oldKeywords == len(keywords)) and counter > 1 and entityCheckCount == 1:
                trainingLoop = False
            
            eval_model()
            
        

        #for element in myTweets:
            #findNewKeywords(element[0], keywords)

        print("Total keywords: ", str(len(keywords)))
        print("Total Tweets: ", str(len(myTweets)))
        print("List of Keywords:\n\n",keywords,"\n\n")
        counter += 1
        
    print('Training Done...')
        
        
        

def information(data):
    myData = {}
    totalEnt=0
    
    for i in range(len(data[0])):
        doc = nlp(preProcess(data[0][i]))
        if len(doc.ents) == 0:
            noEntity.append(preProcess(data[0][i]))
            
        #print(len(doc.ents))
        if(len(doc.ents) == 4):
            print(doc)
            
        for entity in doc.ents: 
        #print(entity.label_)
            totalEnt+=1
            if(entity.label_ == 'FOOD'):
                if entity.text in myData:
                    myData[entity.text] += 1
                else:
                    myData[entity.text] = 1
                    
    print("Number of entities found: " + str(len(myData)))
    print(totalEnt)
    for i in sorted(myData, key = myData.get):
        print("Entity: " + i, "Count: " + str(myData[i]), "Density: " + str(format(myData[i]/totalEnt, '.2f')), end = "\n")
    
    
    return myData




for i in range(len(test_data['tweet'])):
    test_data['tweet'][i] = preProcess(test_data['tweet'][i])



y = test_data['food'].tolist()
print(nlp.pipe_names)
    
def ent_recognize(text):
    doc = nlp(text)
    displacy.render(doc,style = "ent")
    
def predict(tweet):
    doc = nlp(str(tweet))
    if doc.ents:
        displacy.render(doc,style = "ent")

def returnPrediction(tweet):
    nlp = spacy.load(MODEL_PATH)
    doc = nlp(str(tweet))
    if doc.ents:
        return 1
    else:
        return 0
    
def get_predictions():
    predictions = []
    for tweet in test_data['tweet'].tolist():
        predictions.append(returnPrediction(tweet))
    return predictions
    
def eval_model():
    nlp = spacy.load(MODEL_PATH)
    predictions = get_predictions()
    print(metrics.confusion_matrix(y,predictions, labels = [1,0]))
    print(metrics.classification_report(y,predictions, labels = [1,0]))
    
def show_tp():
    counter = 0
    tweets = test_data['tweet'].tolist()
    predictions = get_predictions()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 1:
            print("True positives:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_tn():
    counter = 0
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 0:
            print("True Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fn():
    predictions = get_predictions()
    tweets = test_data['tweet']
    counter = 0
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 1:
            print("False Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fp():
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 0:
            print("False Positive:")
            doc = nlp(str(tweets[i]))
            if doc.ents:
                displacy.render(doc,style = "ent")

[]


# Checking for overlapping words

In [4]:
rankTweet("chicken", None)
# def checkForOverlaps(data):
    
#     print(data)
# keywords = foodKeeperInfo()
# testdata = convertToTrainingFormat("My rice cakes is tasty ", keywords)

# # for word in keywords:
# #     for word2 in wordsInFoodkeeper:
# #         if word in word2 and word != word2:
# #             print(word,word2)



0

## Use the function below to check individual sentences

In [8]:
nlp = spacy.load(MODEL_PATH)
ent_recognize("my friend is chicken because he is scared")

testTweets = live_tweets[5]
for tweet in testTweets[:500]:
    if nlp(preProcess(tweet)).ents:
        ent_recognize(preProcess(tweet))


## Use the function below to check model performance on the entire test set

In [ ]:
#eval_model()

## Use the functions below to see TP, TN, FP, FN respectively

In [2]:
# show_tp()
# show_tn()
# show_fp()
#show_fn()

In [23]:
# foodkeeper = foodKeeperInfo()
# print(foodkeeper)
# sortedKeywords =  sorted(keywordRanker, key=keywordRanker.get, reverse=True)

# for i in range(15): #sortedKeywords
#     keywords.append(sortedKeywords[i])
# print(keywords)
# keywords.append("chicken")
# #keywords.append("cream cheese")

# abc = convertToTrainingFormat("I like to eat cream and cheese with chicken test", keywords)
# print(abc)


# Find the most common words

In [2]:
commonVerbs = getCommonVerbs(training_data) 
#commonVerbs = ['eat', 'know', 'think', 'want', 'got', 'like', 'love', 'use', 'need', 'add']

# Training Loop

In [3]:
trainModel(training_data)
#print(keywords)  


['eat', 'know', 'think', 'want', 'got', 'like', 'love', 'use', 'need', 'add']
Common Verbs gathered... 

~~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~~
Model loaded...
Total keywords:  15
Total Tweets:  0
List of Keywords:

 ['cheese', 'chicken', 'milk', 'butter', 'cream', 'fruit', 'rice', 'water', 'garlic', 'bread', 'sugar', 'salt', 'chocolate', 'fish', 'beef'] 


~~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~~
Model loaded...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 194/194 [00:00<00:00, 1283.66it/s]


Skipping entity
Skipping entity
Skipping entity
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     40.50    0.00    0.00    0.00    0.00
  2     200         54.36   1148.80   97.98   96.86   99.13    0.98
  4     400        136.43    159.72   99.32   98.78   99.88    0.99
  8     600        154.84    106.89   99.81   99.75   99.88    1.00
 12     800         81.31     37.85  100.00  100.00  100.00    1.00
 17    1000         39.90     10.64  100.00  100.00  100.00    1.00
 24    1200         65.72     17.81  100.00  100.00  100.00    1.00
 32    1400        109.73     23.85  100.

 85%|█████████████████████████████████████████████████████████████████████████████▏             | 434/512 [00:00<00:00, 1485.15it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


...

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|███████████████████████████████████████████████████████████████████████████████████████| 12355/12355 [00:06<00:00, 1952.46it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     38.50    0.00    0.00    0.00    0.00
  0     200         72.30   2485.37   74.89   76.97   72.93    0.75
  0     400         66.15   1302.31   85.83   81.75   90.33    0.86
  0     600         69.20   1264.43   87.27   82.45   92.68    0.87
  0     800         78.99   1260.53   90.43   85.88   95.49    0.90
  0    1000         80.60   1535.29   91.60   87.46   96.15    0.92
  0    1200         76.54   1666.45   92.43   87.77   97.61    0.92
  0    1400        104.16   1734.92   93.20   89.37   97.37    0.93
  0    1600        12

[[37  7]
 [ 3 36]]
              precision    recall  f1-score   support

           1       0.93      0.84      0.88        44
           0       0.84      0.92      0.88        39

    accuracy                           0.88        83
   macro avg       0.88      0.88      0.88        83
weighted avg       0.88      0.88      0.88        83

Total keywords:  385
Total Tweets:  12355
List of Keywords:

 ['cheese', 'chicken', 'milk', 'butter', 'cream', 'fruit', 'rice', 'water', 'garlic', 'bread', 'sugar', 'salt', 'chocolate', 'fish', 'beef', 'yogurt', 'almond milk', 'greens', 'granola', 'nuts', 'chia seeds', 'onions', 'apples', 'peanut butter', 'squash', 'soda', 'flour', 'garlic powder', 'buttermilk', 'potatoes', 'cottage cheese', 'sour cream', 'turkey', 'beans', 'cream cheese', 'bagel', 'pretzels', 'celery', 'white wine', 'margarine', 'whipped cream', 'ice cream', 'pineapple', 'whipped topping', 'plantains', 'veal', 'mustard', 'corned beef', 'rye', 'fried chicken', 'gravy', 'tuna', 'v

In [4]:
def getCommonVerbs():
    import en_core_web_sm
    nlp2 = spacy.load("en_core_web_sm")
    count = 0
    myVerbs = {}
    for i in range(10): #(len(training_data)):
        doc = nlp2(training_data[0][i])
        for token in doc:
            print(token, token.pos_)
            if token.pos_ == "VERB":
                if token.text in myVerbs:
                    myVerbs[token.text] = myVerbs[token.text] + 1
                else:
                    if token.text not in nlp2.Defaults.stop_words:
                        myVerbs[token.text] = 1
        
    print(count)
    topVerbs = dict(sorted(myVerbs.items(), key = lambda item: item[1], reverse=True)[:10])
    return [key for key in topVerbs]
print(getCommonVerbs())

@feelsosscatterd PROPN
BILLIE PROPN
IS AUX
BUTTER PROPN
? PUNCT
@coffeespoonie PROPN
Green PROPN
yogurt NOUN
smoothie NOUN
— PUNCT
yogurt PROPN
w/ PROPN
almond PROPN
milk NOUN
, PUNCT
fruit NOUN
, PUNCT
greens NOUN
. PUNCT
Blend PROPN
. PUNCT
Then ADV
add VERB
granola NOUN
, PUNCT
nuts NOUN
, PUNCT
chia ADJ
seeds NOUN
. PUNCT
Smoked ADJ
salmon NOUN
on ADP
sliced PROPN
cucumber PROPN
& CCONJ
amp PROPN
; PUNCT
whole ADJ
wheat NOUN
toast NOUN
w/ NOUN
sliced VERB
raw ADJ
purple NOUN
( PUNCT
red ADJ
) PUNCT
onions NOUN
& CCONJ
amp PROPN
; PUNCT
a DET
dash NOUN
of ADP
lemon NOUN
pepper NOUN
. PUNCT
Apples NOUN
or CCONJ
chocolate VERB
w/ NOUN
peanut NOUN
butter NOUN
. PUNCT
Jarred PROPN
peaches VERB
w/ PROPN
yogurt PROPN
. PUNCT
https://t.co/t7ouQvMU8 PROPN
m VERB
@LuLuRoche PROPN
Or CCONJ
2 X
. PUNCT
Mix VERB
the DET
egg NOUN
in ADP
milk NOUN
. PUNCT
In ADP
the DET
meantime NOUN
crush NOUN
up ADP
soda NOUN
crackers NOUN
with ADP
flour NOUN
. PUNCT
Crush VERB
the DET
crackers NOUN
really ADV


In [3]:
nlp = spacy.load(MODEL_PATH)
myTweet = "My chicken is tasty"
ent_recognize("chicken curry with coconut cream rice cake made with coconut water topped with salted egg sweet bean and candied baby coconut dessert topped with fresh shaved ice leche flan made with duck eggs and lime zest pastillas de leche made with carabao milk")
# x = nlp(myTweet)
# splitSent = myTweet.split()
# for i in range(len(x.ents)):
#     if str(x.ents[i]) in splitSent:
#         print(splitSent.index(str(x.ents[i])))
        
# print(x.doc.ents)
# print(splitSent)

In [4]:
#def findPreviousNextWord(tweets, keywords):
foodKeeperKeywords = foodKeeperInfo()
commonWords = {}
commonPairWords = {}
nlp2 = spacy.load("en_core_web_sm")
for i in range(len(training_data)):
    #Preprocess and remove stop words from each tweet
    myTweet = preProcess(training_data[0][i]).split()
    myTweet = [word for word in myTweet if not word in nlp2.Defaults.stop_words]
    
    for i in range(len(myTweet)):
        if myTweet[i] in foodKeeperKeywords:
            try:
                leftWord = myTweet[i-1]
                if leftWord in commonWords:
                    commonWords[leftWord] += 1
                else:
                    commonWords[leftWord] = 1
            except:
                pass
            try: 
                rightWord = myTweet[i+1]
                if rightWord in commonWords:
                    commonWords[rightWord] += 1
                else:
                    commonWords[rightWord] = 1
            except:
                pass
            try:
                if (leftWord, rightWord) in commonPairWords:
                    commonPairWords[(leftWord,rightWord)] += 1
                else:
                    commonPairWords[(leftWord,rightWord)] = 1
            except:
                pass

#commonWords are words that occur before and after a keyword
commonWords = dict(sorted(commonWords.items(), key = lambda item: item[1], reverse=True))
commonWords = [key for key in commonWords]

#commonPairWords are pairs or words that occur before and after a keyword
commonPairWords = dict(sorted(commonPairWords.items(), key = lambda item: item[1], reverse=True))
#print(training_data[0][1000])
#print(foodKeeperKeywords)

In [ ]:
#print(commonWords[:25])
myIter= iter(commonPairWords)
for iter1 in iter(commonPairWords):
    print(iter1, commonPairWords[iter1])

In [80]:
import re
#count all words and adjacent
def findAdjacentWords(data,keywords):
    allWords = {}
    count = 0
    for tweet in data[0][:10]:
        splitTweet = preProcess(tweet).split()
        for i in range(len(splitTweet)):
            word = splitTweet[i]
            try:
                biword = word + " " + splitTweet[i + 1]
                if biword in keywords:
                    print("Biword:", biword)
                    continue
            except:
                pass
            try:
                triword = word + " " + splitTweet[i + 1] + " " + splitTweet[i + 2]
                if triword in keywords:
                    print("Triword:", triword)
                    continue
            except:
                pass
            
        
    
    '''mySent = "I am eating chicken for breakfast lunch and dinner tonight."
    mySearch = re.search('chicken', mySent)
    mySpan = mySearch.span()
    print(mySent[mySpan[0]:mySpan[1]])
    '''
findAdjacentWords(training_data, foodKeeperKeywords)

Biword: almond milk
Biword: chia seeds
Biword: peanut butter
Biword: peanut butter
Biword: cashew butter


# Test Data with 1.6 Million Tweet Dataset

In [13]:
# print(live_tweets[5][:10])
print(test_data['tweet'])

0     just microwaved a kashi chicken and spinach th...
1      thats really sad i wolud hate that! but i had...
2      and it took me my entire walk to the train st...
3     just finished cooking spag bol from scratch in...
4     oh noooooo kath is back from annual leave!!!!!...
                            ...                        
78    sick roomie gave me her cold my throats sore (...
79    no flying to ponca city today for breakfast oa...
80    just walked by marksandspencers food n didn't ...
81    if lucas till and taylor swift start dating i ...
82                             bed time back to reality
Name: tweet, Length: 83, dtype: object


# See what keywords are found by the created model

In [15]:
keywordsFound = []
nlp = spacy.load(MODEL_PATH)
# print(nlp.pipeline)
for tweet in live_tweets[5]: #test_data['tweet']:
    modeledTweet = nlp(preProcess(tweet))
    for token in modeledTweet.doc.ents:
        if str(token) in keywordsFound: continue
        keywordsFound.append(str(token))

# If the model finds food keywords print the Tweet
To help visualize which keywords are being found this loop iterates through the Tweets testing the model to see what keywords it finds. If it finds a keyword in the Tweet it will print the Tweet and highlight the keyword

In [14]:
for tweet in live_tweets[5]: #test_data['tweet'][:500]:
    ents = nlp(preProcess(tweet))
    #if ents.doc.ents:
    ent_recognize(preProcess(tweet))

KeyboardInterrupt: 

# Old Results (Don't Change Unless on Purpose!)

In [ ]:
foodK1 = foodKeeperInfo()
notInFoodKeeper = []
inFoodKeeper = []
for keyword in keywordsFound:
    if keyword not in foodK1 and keyword not in notInFoodKeeper: notInFoodKeeper.append(keyword)
    elif keyword in foodK1 and keyword not in inFoodKeeper: inFoodKeeper.append(keyword)
        
print(len(notInFoodKeeper), len(inFoodKeeper))
print(inFoodKeeper)
#Normal Process

In [16]:
foodK1 = foodKeeperInfo()
notInFoodKeeper = []
inFoodKeeper = []
for keyword in keywordsFound:
    if keyword not in foodK1 and keyword not in notInFoodKeeper: notInFoodKeeper.append(keyword)
    elif keyword in foodK1 and keyword not in inFoodKeeper: inFoodKeeper.append(keyword)
        
print(len(notInFoodKeeper), len(inFoodKeeper))

10443 316
